# Розмітка частин мови (Parts-of-Speech Tagging)

У цій лабораторній роботі Ви познайомитеся з методом вирішення задачі розмітки частин мови (POS), а саме процесу присвоєння тегів частин мови (іменник, прикметник та ін.) словам у текстовому корпусі, за допомогою прихованої марковської моделі.

Розмітка частин мови у тексті є доволі важливою для побудови високоточних пошукових систем, чат-ботів і діалогових агентів, а також алгоритмів автоматичного виправлення тексту.


## Завантаження вихідних даних та програмного коду (опціонально)

Для виконання цієї роботи необхідно мати повний архів, що містить файли Python з допоміжними функціями та окрему папку з підготовленим датасетом.

Якщо у Вас з якихось причин є тільки цей ноутбук, Ви можете завантажити усі необхідні файли з GitHub за допомогою наступних команд:

In [1]:
!wget https://github.com/niksyromyatnikov/opnu-ml-assignments/archive/refs/heads/main.zip && unzip main.zip

--2024-07-22 00:03:22--  https://github.com/niksyromyatnikov/opnu-ml-assignments/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/niksyromyatnikov/opnu-ml-assignments/zip/refs/heads/main [following]
--2024-07-22 00:03:22--  https://codeload.github.com/niksyromyatnikov/opnu-ml-assignments/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [  <=>               ]   2.46M  5.31MB/s    in 0.5s    

2024-07-22 00:03:23 (5.31 MB/s) - ‘main.zip’ saved [2583327]

Archive:  main.zip
71d4ee037585e19d1071ccd131e8578a76080d20
   creating: opnu-ml-assignments-ma

In [2]:
cd opnu-ml-assignments-main/part-of-speech-tagging-with-hmm

/content/opnu-ml-assignments-main/part-of-speech-tagging-with-hmm


## Підготовка середовища

In [3]:
# Встановлення залежностей
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 30.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [4]:
# Імпорт бібліотек
import pandas as pd
import numpy as np

import math
from typing import Dict, List, Tuple

from data_helper import load_data
from hmm_helper import calculate_dicts, build_transitions, build_emissions
from eval_helper import calculate_accuracy

<a name='0'></a>
## Вихідні дані
У цій лабораторній роботі використовується розмічений набір даних **Wall Street Journal (WSJ)**.
Опис та приклади наявних у датасеті тегів (частин мови) можна переглянути за [посиланням](http://relearn.be/2015/training-common-sense/sources/software/pattern-2.6-critical-fork/docs/html/mbsp-tags.html).


Тренувальний набір `train_pos.txt` використовується для побудови матриць переходів та виходів, а також для обчислення кількості входжень тегів.

Словник `vocab.txt` містить унікальні слова, що зустрічаються у датасеті більше одного разу.

Тестовий набір зі словами та тегами наведено у `test_pos.txt`, він використовується для оцінки точності передбачення.

Для передбачення використовується окремий файл `test_predict.txt`, що представляє собою тестовий набір без тегів.

In [5]:
# Завантаження вихідних даних
train_corpus, y, vocab, prep = load_data()

Перші 20 рядків тренувального набору:
['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n', 'of\tIN\n', '``\t``\n', 'The\tDT\n', 'Misanthrope\tNN\n', "''\t''\n", 'at\tIN\n', 'Chicago\tNNP\n', "'s\tPOS\n", 'Goodman\tNNP\n', 'Theatre\tNNP\n', '(\t(\n', '``\t``\n', 'Revitalized\tVBN\n', 'Classics\tNNS\n', 'Take\tVBP\n']


Перші 20 елементів з файлу словника: ['!', '#', '$', '%', '&', "'", "''", "'40s", "'60s", "'70s", "'80s", "'86", "'90s", "'N", "'S", "'d", "'em", "'ll", "'m", "'n'"]
Останні 20 елементів з файлу словника: ['youngsters', 'your', 'yourself', 'youth', 'youthful', 'yuppie', 'yuppies', 'zero', 'zero-coupon', 'zeroing', 'zeros', 'zinc', 'zip', 'zombie', 'zone', 'zones', 'zoning', '{', '}', '']

Перші 20 елементів словника:
Ключ: Значення
: 0
!: 1
#: 2
$: 3
%: 4
&: 5
': 6
'': 7
'40s: 8
'60s: 9
'70s: 10
'80s: 11
'86: 12
'90s: 13
'N: 14
'S: 15
'd: 16
'em: 17
'll: 18
'm: 19
'n': 20


Перші 20 рядків тестового набору:
['The\tDT\n', 'economy\tNN\n', "'s\tPOS\n", 'tempe

<a name='1'></a>
# 1. Розмітка частин мови: Тренування

#### Число переходів
- `transition_counter` - словник, який обчислює, скільки разів кожен тег зустрічається після іншого тегу.

Він використовується для обчислення ймовірності появи тегу на позиції $i$ з урахуванням тегу на позиції $i-1$:
$$P(t_i |t_{i-1}) \tag{1}$$

#### Число виходів

`emission_counter` - словник, який обчислює, скільки разів кожне слово зустічається для кожного тегу.

Він використовується для обчислення ймовірності появи слова з урахуванням його тегу (виходу тегів у слова):
$$P(w_i|t_i)\tag{2}$$

#### Число тегів

`tag_counter` - словник, де ключем є тег, а значенням - кількість його появ у текстовому корпусі.

In [6]:
emission_counter, transition_counter, tag_counter = calculate_dicts(train_corpus, vocab)

оброблено 100000 елементів
оброблено 200000 елементів
оброблено 300000 елементів
оброблено 400000 елементів
оброблено 500000 елементів
оброблено 600000 елементів
оброблено 700000 елементів
оброблено 800000 елементів
оброблено 900000 елементів


In [7]:
states = sorted(tag_counter.keys())
print(f"Число тегів (частин мови): {len(states)}")
print(f"Список тегів (частин мови): {states}")

Число тегів (частин мови): 46
Список тегів (частин мови): ['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [8]:
limit = 20
print("Приклад переходів у словнику transition_counter:")
for idx, tc in enumerate(transition_counter.items()):
    if idx == limit:
        break
    print(tc)

print("\n\nПриклад виходів у словнику emission_counter:")
for idx, ec in enumerate(emission_counter.items()):
    if idx == limit:
        break
    print(ec)

Приклад переходів у словнику transition_counter:
(('--s--', 'IN'), 5050)
(('IN', 'DT'), 32364)
(('DT', 'NNP'), 9044)
(('NNP', 'CD'), 1752)
(('CD', 'NN'), 7377)
(('NN', 'IN'), 32885)
(('IN', '``'), 546)
(('``', 'DT'), 1014)
(('DT', 'NN'), 38873)
(('NN', "''"), 686)
(("''", 'IN'), 591)
(('IN', 'NNP'), 14753)
(('NNP', 'POS'), 5094)
(('POS', 'NNP'), 905)
(('NNP', 'NNP'), 34465)
(('NNP', '('), 313)
(('(', '``'), 42)
(('``', 'VBN'), 75)
(('VBN', 'NNS'), 759)
(('NNS', 'VBP'), 5076)


Приклад виходів у словнику emission_counter:
(('IN', 'In'), 1735)
(('DT', 'an'), 3142)
(('NNP', 'Oct.'), 317)
(('CD', '19'), 100)
(('NN', 'review'), 36)
(('IN', 'of'), 22925)
(('``', '``'), 6967)
(('DT', 'The'), 6795)
(('NN', 'Misanthrope'), 3)
(("''", "''"), 6787)
(('IN', 'at'), 4361)
(('NNP', 'Chicago'), 197)
(('POS', "'s"), 8079)
(('NNP', 'Goodman'), 7)
(('NNP', 'Theatre'), 5)
(('(', '('), 1153)
(('VBN', '--unk_upper--'), 93)
(('NNS', '--unk_upper--'), 350)
(('VBP', 'Take'), 1)
(('DT', 'the'), 41098)


<a name='2'></a>
# 2. Прихована марковська модель для розмітки частин мови

Марковська модель містить ряд станів і ймовірностей переходу між цими станами (частинами мови).
- Марковська модель використовує матрицю переходів `T`.
- Прихована марковська модель додає матрицю виходів `E`, що описує ймовірність виходу до спостереження (слова), коли ми перебуваємо в певному стані (частина мови).

<a name='2.1'></a>
## 2.1 Побудова матриць

### Побудова матриці переходів T

Нижче наведено приклад спрощеної до 5 тегів матриці переходів T.
Кожна комірка матриці містить обчислену ймовірність переходу від одного тегу (частини мови) до іншого.
Для кожної частини мови (рядок) наступна обирається із доступних стовпців, тому сума ймовірностей кожного рядка має дорівнювати 1.

|**T**    |....|         NN   |          JJ  |         TO   |      VB      |          .  | ...|
|---------|----|--------------|--------------|--------------|--------------|-------------|----|
|**NN**   | ...| 0.122172     | 0.008786     | 0.039538     | 0.001399     | 0.109023    | ...|
|**JJ**   | ...| 0.450038     | 0.074669     | 0.027884     | 0.000114     | 0.024389    | ...|
|**TO**   | ...| 0.032831     | 0.030908     | 0.000090     | 0.574808     | 0.000760    | ...|
|**VB**   | ...| 0.061616     | 0.084651     | 0.043384     | 0.004917     | 0.024208    | ...|
|**.**    | ...| 0.000329     | 0.000101     | 0.000025     | 0.000177     | 0.000177    | ...|
| ...     | ...|     ...      |     ...      |     ...      |     ...      |     ...     | ...|


Для обчислення значень матриці використовується наступна формула:

$$ P(t_i | t_{i-1}) = \frac{C(t_{i-1}, t_{i}) + \alpha }{C(t_{i-1}) +\alpha * N}\tag{3}$$

- $N$ - загальна кількість тегів
- $C(t_{i-1}, t_{i})$ - кількість появ переходу (попередній тег, поточний тег) у словнику `transition_counter`.
- $C(t_{i-1})$ - кількість появ попереднього тегу у словнику `tag_counter`.
- $\alpha$ - параметр згладжування.

In [9]:
T = build_transitions(transition_counter, tag_counter)

state_names = ["NN", "JJ", "TO", "VB", "."]
states_idxs = [states.index(x) for x in state_names]

print("Часткова матриця переходів T")
T_sub = pd.DataFrame(np.vstack([T[idx, states_idxs] for idx in states_idxs]), index=state_names, columns=state_names)
print(T_sub)

Часткова матриця переходів T
          NN        JJ        TO        VB         .
NN  0.122172  0.008786  0.039538  0.001399  0.109023
JJ  0.450038  0.074669  0.027884  0.000114  0.024389
TO  0.032831  0.030908  0.000090  0.574808  0.000760
VB  0.061616  0.084651  0.043384  0.004917  0.024208
.   0.000329  0.000101  0.000025  0.000177  0.000177


### Побудова матриці виходів E

Матриця виходів E має розмірність (num_tags, N), де num_tags - число можливих тегів (частин мови)

Нижче наведено приклад спрощеної до 5 слів матриці виходів E.

|**E**   | ...|  statistical   |        model   |    predicted    |     word        |           .    | ...|
|----    |----|----------------|----------------|-----------------|-----------------|----------------|----|
|**NN**  | ...| 7.521128e-09   |**3.459794e-04**| 7.521128e-09    |**3.384583e-04** | 7.521128e-09   | ...|
|**JJ**  | ...|**1.959642e-04**| 3.267431e-05   | 1.632899e-08    | 1.632899e-08    | 1.632899e-08   | ...|
|**TO**  | ...| 4.468120e-08   | 4.468120e-08   | 4.468120e-08    | 4.468120e-08    | 4.468120e-08   | ...|
|**VB**  | ...| 3.779036e-08   | 3.782815e-05   | 3.779036e-08    | 3.779036e-08    | 3.779036e-08   | ...|
|**VBD** | ...| 3.343053e-08   | 3.343053e-08   |**1.537838e-03** | 3.343053e-08    | 3.343053e-08   | ...|
|**.**   | ...| 2.531532e-08   | 2.531532e-08   | 2.531532e-08    | 2.531532e-08    |**9.878037e-01**| ...|
| ...    | ...|     ...        |     ...        |     ...         |     ...         |     ...        | ...|

Для обчислення значень матриці використовується наступна формула:

$$P(w_i | t_i) = \frac{C(t_i, word_i)+ \alpha}{C(t_{i}) +\alpha * N}\tag{4}$$

- $C(t_i, word_i)$ - число разів, коли слово $word_i$ зустрічалось з тегом $tag_i$ в тренувальному наборі.
- $C(t_i)$ - число разів, коли тег $tag_i$ зустрічався у тренувальному наборі.
- $N$ - число слів у словнику.
- $\alpha$ - параметр згладжування.

In [10]:
E = build_emissions(emission_counter, tag_counter, list(vocab))

# Демонстрація ймовірностей виходів матриці E для обраних слів та тегів
words_to_check  = ["statistical", "model", "predicted", "word", "."]
cols = [vocab[a] for a in words_to_check]
row_names =["NN", "JJ", "TO", "VB", "VBD", "."]
rows = [states.index(a) for a in row_names]
E_sub = pd.DataFrame(E[np.ix_(rows,cols)], index=row_names, columns=words_to_check )
print(E_sub)

      statistical         model     predicted          word             .
NN   7.521128e-09  3.459794e-04  7.521128e-09  3.384583e-04  7.521128e-09
JJ   1.959642e-04  3.267431e-05  1.632899e-08  1.632899e-08  1.632899e-08
TO   4.468120e-08  4.468120e-08  4.468120e-08  4.468120e-08  4.468120e-08
VB   3.779036e-08  3.782815e-05  3.779036e-08  3.779036e-08  3.779036e-08
VBD  3.343053e-08  3.343053e-08  1.537838e-03  3.343053e-08  3.343053e-08
.    2.531532e-08  2.531532e-08  2.531532e-08  2.531532e-08  9.878037e-01


<a name='3'></a>
# 3. Алгоритм Вітербі

Реалізація алгоритму Вітербі, що є алгоритмом динамічного програмування, складається з трьох основних етапів:

* **Ініціалізація** - на цьому етапі ініціалізуються матриці `best_paths` та `best_probabilities`, що потім використовуються для прямого проходження.
* **Пряме проходження** - на кожній ітерації прямого проходження обчислюється ймовірність кожного шляху (послідовність тегів) та найкращих шляхів до цієї ітерації.
* **Зворотне проходження** - цей етап використовується для пошуку найкращого (найімовірнішого) шляху (послідовності тегів).

<a name='3.1'></a>
## 3.1 Завдання: імплементація етапу ініціалізації

Необхідно ініціалізувати дві матриці однакової розмірності.

- `best_probs`: кожна комірка містить ймовірність переходу від одного тегу (частини мови) до слова у корпусі.

- `best_paths`: матриця, яка використовується для визначення найкращого можливого шляху.

Обидві матриці потрібно заповнити нулями, за винятком нульового стовпця `best_probs`.
Нульовий стовпець `best_probs` заповнюється з припущенням, що першому слову корпусу передує початковий токен ("--s--").

Ініціалізація нульового стовпця `best_probs` відбувається наступним чином:
- Імовірність найкращого шляху від початкового індексу до заданого тегу з індексом $i$ позначається як best_probs$[s_{idx}, i]$.
- Це оцінюється як ймовірність того, що початковий тег переходить до тегу з індексом $i$ $\mathbf{T}[s_{idx}, i]$ і що тег з індексом $i$ виходить у перше слово корпусу $ \mathbf{E}[i, vocab[corpus[0]]] $.

Математична форма запису:
best_probs$[s_{idx}, i] = \mathbf{T}[s_{idx}, i] \times \mathbf{E}[i, corpus[0] ]$


Для уникнення множення та малих значень ми беремо логарифм добутку, що стає сумою двох логарифмів:

best_probs$[i, 0] = log(T[s_{idx}, i]) + log(E[i, vocab[corpus[0]]])$

Крім того, щоб уникнути логарифм 0, best_probs$[i, 0] = float('-inf')$ коли $T[s_{idx}, i] == 0$

<br>

Отже, реалізація ініціалізації `best_probs` має такий вигляд:

Якщо $T[s_{idx}, i] <> 0$ тоді best_probs$[i,0] = log(T[s_{idx}, i]) + log(E[i, vocab[corpus[0]]])$

Якщо $T[s_{idx}, i] == 0$ тоді best_probs$[i,0] = float('-inf')$


<br>

*Для логарифму використовуйте [math.log](https://docs.python.org/3/library/math.html)

**Представляйте нескінченність і негативну нескінченність так:

```CPP
float('inf')
float('-inf')
```

In [11]:
def initialize(
    states: List[str],
    tag_counter: Dict[str, int],
    T: List[List[float]],
    E: List[List[float]],
    corpus: List[str],
    vocab: Dict[str, int]
) -> Tuple[List[List[float]], List[List[int]]]:
    """
    states: список усіх тегів (частин мови)
    tag_counter: словник, де тег є ключем, а число його появи у наборі - значенням
    T: матриця переходів розмірністю tags_total x tags_total
    E: матриця виходів розмірністю (tags_total, len(vocab))
    corpus: послідовність слів для ідентифікації тегів
    vocab: словник, де слово є ключем, а індекс - значенням

    proba_best: матриця розмірністю (tags_total, len(corpus))
    path_best: матриця розмірністю (tags_total, len(corpus))
    """
    
    # Реалізуйте функцію тут

    return proba_best, path_best

In [12]:
proba_best, path_best = initialize(states, tag_counter, T, E, prep, vocab)

In [ ]:
print(f"proba_best[0,0]: {proba_best[0, 0]:.4f}")
print(f"path_best[2,3]: {path_best[2, 3]:.4f}")

<a name='3.2'></a>
## 3.2 Завдання: імплементація етапу прямого проходження

У цьому завданні необхідно реалізувати логіку заповнення матриць `best_probs` та `best_paths`:

Формули для обчислення ймовірності та шляху для слова з індексом $i$ у $corpus$:

$\mathrm{prob} = \mathbf{best\_prob}[k, i-1] + \mathrm{log}(\mathbf{T}[k, j]) + \mathrm{log}(\mathbf{E}[j, vocab[corpus[i] ] ) ]$

де $i-1$ - індекс попереднього слова у корпусі, $j$ - індекс поточного тегу, а $k$ - індекс попереднього тегу

$\mathrm{path} = k$

де $k$ — ціле число, що представляє попередній тег.

<br>

Реалізуйте етап прямого проходження `forward()` для обчислення `best_path` і `best_prob` для кожного тегу і кожного слова, використовуючи наведений нижче псевдокод.

```
для кожного слова у корпусі

    для кожного тегу_j (частини мови), що може зустрічатися з цим словом
    
        для кожного тегу_k, що може зустрічатися з попереднім словом

            обчислити ймовірність того, що попереднє слово мало тег_k, поточне слово має тег_j, а також тег_j виходить у поточне слово
            
            зберегти найбільшу ймовірність обчислену для поточного слова

            занести найбільшу ймовірність у матрицю `best_probs`

            занести індекс `k`, що ідентифікує тег попереднього слова та з яким було отримано найбільшу ймовірність, у матрицю `best_paths`
```

<br>

*Для логарифму використовуйте [math.log](https://docs.python.org/3/library/math.html)

In [14]:
def forward(
    T: List[List[float]],
    E: List[List[float]],
    test_corpus: List[str],
    proba_best: List[List[float]],
    path_best: List[List[float]],
    vocab: Dict[str, int]
) -> Tuple[List[List[float]], List[List[int]]]:
    """
    T: матриця переходів розмірністю tags_total x tags_total
    E: матриця виходів розмірністю (tags_total, len(vocab))
    test_corpus: підготовлений тестовий корпус у вигляді списку cлів
    proba_best: ініціалізована матриця розмірністю (tags_total, len(corpus))
    path_best: ініціалізована матриця розмірністю (tags_total, len(corpus))
    vocab: словник, де слово є ключем, а індекс - значенням

    proba_best: ініціалізована матриця розмірністю (tags_total, len(corpus))
    path_best: ініціалізована матриця розмірністю (tags_total, len(corpus))
    """
    
    # Реалізуйте функцію тут

    return proba_best, path_best

In [ ]:
proba_best, path_best = forward(T, E, prep, proba_best, path_best, vocab)

In [ ]:
print(f"proba_best for word {prep[1]}:")
for tag, proba in zip(states, proba_best[:, 1].tolist()):
    print(tag, proba)

<a name='3.3'></a>
## 3.3 Завдання: імплементація зворотного поширення

У цьому завданні необхідно імплементувати метод зворотного поширення `backward()` для алгоритму Вітербі, що для кожного слова з текстового корпусу передбачує тег (частину мови) за допомогою обчислених раніше матриць `best_paths` та `best_probs`.

Метод складається з двох кроків:
1. Пройти кожен рядок (тег) для останнього стовпця (останнього слова у корпусі) матриці `best_probs` та визначити індекс рядку (тегу) з найбільшим значенням. Конвертувати індекс у тег та зберегти його у списку передбачення.
2. Знайти найімовірніший тег для попереднього слова за допомогою отриманого індексу для поточного (останнього) слова та матриці `best_paths`. Записати отриманий тег у список передбачень та повторити крок для усіх попередніх слів.

In [17]:
def backward(
    proba_best: List[List[float]],
    path_best: List[List[float]],
    corpus: List[str],
    states: List[str]
) -> List[str]:
    """
    proba_best: ініціалізована матриця розмірністю (tags_total, len(corpus))
    path_best: ініціалізована матриця розмірністю (tags_total, len(corpus))
    corpus: підготовлений тестовий корпус у вигляді списку cлів
    states: список усіх тегів (частин мови)

    pred: список тегів розмірністю len(corpus), де кожний тег є передбаченням
    для відповідного слова у корпусі
    """
    
    # Реалізуйте функцію тут

    return pred

In [ ]:
pred = backward(proba_best, path_best, prep, states)

limit = 200
curr_i = 0
print("Word | True | Pred")
for ground_truth, pred_tag in zip(y, pred):
    if curr_i > limit:
        break
    curr_i += 1
    word_and_tag = ground_truth.replace('\n', '').split('\t')
    print(word_and_tag[0], '|', word_and_tag[1] if len(word_and_tag) == 2 else ' --s--', '|', pred_tag)

<a name='4'></a>
# 4. Оцінка точності

Обчисліть точність свого прогнозу, порівнявши передбачення `pred` з правильними тегами `y`.

In [ ]:
print(f"Точність: {calculate_accuracy(pred, y):.4f}")